In [1]:
# Auto-install required libraries into the current Python environment (preferably your .venv)
import sys
import subprocess
import importlib

# Map module import name -> pip package name
required = [
    ("pandas", "pandas"),
    ("numpy", "numpy"),
    ("matplotlib", "matplotlib"),
    ("seaborn", "seaborn"),
]

missing = []
for mod, pkg in required:
    try:
        importlib.import_module(mod)
    except Exception:
        missing.append(pkg)

if missing:
    print("Installing missing packages into:", sys.executable)
    print("Packages:", " ".join(missing))
    # Install into the interpreter backing this kernel
    subprocess.run([sys.executable, "-m", "pip", "install", *missing], check=True)
    print("Install complete. You may need to restart the kernel if imports still fail.")
else:
    print("All required packages are already installed.")


All required packages are already installed.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

test = pd.read_csv("../data/test.csv")
train = pd.read_csv("../data/train.csv")

In [3]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
train.drop_duplicates(inplace=True)
test.drop_duplicates(inplace=True)


train = train.drop(columns=["Cabin"])
test = test.drop(columns=["Cabin"])

train.dropna(inplace=True)
test.dropna(inplace=True)

In [5]:
train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64